<a href="https://colab.research.google.com/github/fjcorreiaa/nlp_sentiment/blob/main/analise_de_sentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregando as bibliotecas

In [4]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [8]:
import pandas as pd
import numpy as np
import random
import datasets
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import pipeline
from transformers import DataCollatorWithPadding
import torch
from torch import nn
from sklearn.metrics import f1_score
import tensorflow as tf


# Importando o datasets

In [12]:
#Importando base de treino
df_treino = pd.read_csv ('https://raw.githubusercontent.com/fjcorreiaa/nlp_sentiment/main/Train50.csv', sep=";", encoding='latin-1')
df_treino.head()

#Importando base de testes
df_teste = pd.read_csv ('https://raw.githubusercontent.com/fjcorreiaa/nlp_sentiment/main/Test.csv', sep=";", encoding='latin-1')
df_teste.head()


,id,tweet_text,tweet_date,sentiment,query_used
0,1029536486021099522,@Gazo1a Nossa! Muito obrigada :),Wed Aug 15 01:13:20 +0000 2018,1,:)
1,1029536496368406528,@BerzGamer vai pa puta que te pariu :),Wed Aug 15 01:13:23 +0000 2018,1,:)
2,1029536531655131137,QUER MAIS DESCONTOS? (14/08) â­â­â­â­â­ ð...,Wed Aug 15 01:13:31 +0000 2018,1,:)
3,1029536560117678081,"EU VOU PEGAR VCS, ME AJUDEM GALERA, PELO AMOR ...",Wed Aug 15 01:13:38 +0000 2018,1,:)
4,1029536605852377088,EstÃ¡vamos em casa do ZÃ© e eu estava a morrer...,Wed Aug 15 01:13:49 +0000 2018,1,:)


In [ ]:
#Removendo colunas indesejadas
df_teste = df_teste.drop(columns=['tweet_date', 'id'])
df_treino.head()


df_treino = df_treino.drop(columns=['tweet_date', 'id'])
df_treino.head()

,tweet_text,sentiment,query_used
0,"@bejusmila Ã© um aplicativo a parte, o youtube...",1,:)
1,@Alixandriiaa No problem! :) ^tm,1,:)
2,"@FellipeC @DoctorClimao NÃ£o vejo, pelo menos ...",1,:)
3,@b4rbixando @barbixas Lembra das sÃ¡bias palav...,1,:)
4,ola obgzinho querido :) https://t.co/osihldty12,1,:)


In [ ]:
#Verificando o percentual de amostra em cada categoria
df_treino ["sentiment"].value_counts(normalize=True).sort_index()

0    0.445
1    0.555
Name: sentiment, dtype: float64

In [ ]:
#Criando amostra de treino para otimização computacional
n_samples = len(df_treino)
sample_size = 200
random_indices = random.sample(range(n_samples), sample_size)
df_treino = df_treino.iloc[random_indices]


In [ ]:
#Criando amostra de teste para otimização computacional
n_samples = len(df_teste)
sample_size = 2000
random_indices = random.sample(range(n_samples), sample_size)
df_teste = df_teste.iloc[random_indices]

In [ ]:
#Transformando o dataframe em um dataset pandas
dataset_treino = Dataset.from_pandas(df_treino, preserve_index=False)
dataset_teste = Dataset.from_pandas(df_teste, preserve_index=False)


In [ ]:
dataset_teste


Dataset({
    features: ['tweet_text', 'sentiment', 'query_used'],
    num_rows: 2000
})

In [ ]:
dataset_treino

Dataset({
    features: ['tweet_text', 'sentiment', 'query_used'],
    num_rows: 200
})

# Gerando Tokens

In [ ]:
model_ckpt = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
tokenizer (dataset_treino["tweet_text"] [:1])

{'input_ids': [[101, 1037, 19538, 2050, 10927, 18107, 12436, 2072, 2033, 1997, 7869, 17119, 1037, 11937, 26302, 3351, 2213, 1024, 1007, 1007, 1007, 1007, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
def tokenize_function (sample):
  return tokenizer (sample["tweet_text"])

In [ ]:
tokenized_dataset_treino = dataset_treino.map(tokenize_function)


In [ ]:
tokenized_dataset_teste = dataset_teste.map(tokenize_function)

In [ ]:
tokenized_dataset_treino


Dataset({
    features: ['tweet_text', 'sentiment', 'query_used', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

In [ ]:
tokenized_dataset_teste

Dataset({
    features: ['tweet_text', 'sentiment', 'query_used', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

In [ ]:
#Ajustando tamanho da sequencia de tokens
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors="tf")

##### Transformado dados em um banco do tipo TensorFlow

In [ ]:
tf_train_dataset = tokenized_dataset_treino.to_tf_dataset(
    columns = ["attention_mask", "input_ids", "token_type_ids"],
    label_cols= ["sentiment"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
tf_val_dataset = tokenized_dataset_teste.to_tf_dataset(
    columns = ["attention_mask", "input_ids", "token_type_ids"],
    label_cols= ["sentiment"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

### Iniciando o treinamento do modelo - seleção de modelo pré treinado





In [ ]:
from transformers.onnx.features import TFAutoModelForSequenceClassification

model_name = "roberta-base"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
from transformers.models import opt

opt= tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(
    optimizer = opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124055040 
 r)                                                              
                                                                 
 classifier (TFRobertaClass  multiple                  592130    
 ificationHead)                                                  
                                                                 
Total params: 124647170 (475.49 MB)
Trainable params: 124647170 (475.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________



# Treinando o modelo



In [ ]:
model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=1)

25/25 [==============================] - 1310s 53s/step - loss: 0.6925 - accuracy: 0.5200 - val_loss: 0.6813 - val_accuracy: 0.4965


In [ ]:
preds = model.predict(tf_val_dataset, verbose=1)

250/250 [==============================] - 1002s 4s/step


In [ ]:
preds['logits']

array([[-0.29153857,  0.02457766],
       [-0.24848114,  0.00494554],
       [-0.17721346, -0.03136808],
       ...,
       [-0.20318668, -0.0003577 ],
       [-0.1583144 , -0.06362587],
       [-0.17662334, -0.06675926]], dtype=float32)

In [ ]:
#Transformando logits em probabilidades
tf.nn.softmax(preds['logits']).numpy()

array([[0.4216225 , 0.5783774 ],
       [0.43698025, 0.56301975],
       [0.4636032 , 0.5363969 ],
       ...,
       [0.4494658 , 0.55053407],
       [0.47634548, 0.5236544 ],
       [0.47256157, 0.52743846]], dtype=float32)

In [ ]:
print(np.argmax(preds))

0


# Salvando o modelo

In [ ]:
model.save_weights('model.hdf5')

In [ ]:
#Verificando classificação de novas entradas
model_ckpt ="josu/roberta-pt-br"
pipe = pipeline("text-classification", model=model_ckpt)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at josu/roberta-pt-br and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pipe("Estou muito feliz hoje")

[{'label': 'LABEL_0', 'score': 0.5170710682868958}]

In [ ]:
new_model=TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti